In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

# Numerical Differentiation and Integration
## Lecture 12

### First Derivatives

Approximations to a derivative can be obtained by expanding a function using Taylor series. Here are three schemes for calculating numerically the derivative of a continuous functions. These are called the forward, backward and centered difference schemes. 

#### Forward scheme

$$\frac{df}{dx}(x_0) \approx \frac{f(x_0+\Delta x )-f(x_0)}{\Delta x}$$

#### Backward scheme

$$\frac{df}{dx}(x_0) \approx \frac{f(x_0) - f(x_0 - \Delta x )}{\Delta x}$$

#### Centered scheme
$$\frac{df}{dx}(x_0) \approx \frac{f(x_0+\Delta x)-f(x_0-\Delta x)}{2 \Delta x}$$

Here's how these schemes can be coded

In [ ]:
# Forward difference scheme
def NDf(f, x0, dx):
    return(f(x0+dx) - f(x0)) / dx

# Backward difference scheme 
def NDb(f, x0, dx):
    return(f(x0) - f(x0-dx)) / dx   

# Centered difference scheme 
def NDc(f, x0, dx):
    return (f(x0+dx) - f(x0-dx)) / (2*dx)

Let's define a function that can be derived analytically, for example 
 
$$\frac{d}{dx}\sin(x) = \cos(x)$$

In [ ]:
fig, axes = plt.subplots(figsize=(8,6))
x = np.arange(0, 2*np.pi, 0.1)
y = np.sin(x)
plt.plot(x, np.sin(x))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()

and let's compare the derivative at  $\pi/4$ of the 3 schemes, using say  $\Delta x =0.1$, with the exact solution,

In [ ]:
x0 = np.pi/4
dx = 0.01

print("Exact:", np.cos(x0))
print("  NDf:", NDf(np.sin, x0, dx))
print("  NDb:", NDb(np.sin, x0, dx))
print("  NDc:", NDc(np.sin, x0, dx))

Would be more illuminating to look at the absolute relative error. 

$$ \text{RelativeError} = \frac{\left|\text{ApproxValue} - \text{ExactValue}\right|}{\text{ExactValue}}$$

Let's do this

In [ ]:
exact = np.cos(x0)
print("  NDf: {:.6f}".format(abs(NDf(np.sin, x0, dx) - exact) / exact))
print("  NDb: {:.6f}".format(abs(NDb(np.sin, x0, dx) - exact) / exact))
print("  NDc: {:.6f}".format(abs(NDc(np.sin, x0, dx) - exact) / exact))

We see that the forward and backward schemes have a similar error but that the centered scheme is much more accurate. 

### Second derivatives

The second order derivative can be constructed from the forward and backward scheme and is given by

#### Centered scheme for 2nd order derivative
$$\frac{d^2 f}{dx^2}(x_0) \approx \frac{f(x_0+\Delta x)-2f(x_0)+f(x_0-\Delta x)}{\Delta x^2} $$

Which can be coded like this:

In [ ]:
def NDc2(f, x0, dx):
    return (f(x0+dx) -2*f(x0)+ f(x0-dx)) / (dx**2)

And compared with the exact solution

In [ ]:
x0 = np.pi/4
dx = 0.01
print("Exact:", -np.sin(x0))
print(" NDc2:", NDc2(np.sin, x0, dx))


## Numerical differentiation of discrete functions

Numerical differentiation is largely used for estimating derivatives from experimental data. Consider the following set of measurements of a ball thrown in the air. The list of pairs give the time-position of a ball.

In [ ]:
BallData = np.array( [ 
            [0,1.33],
            [0.03333,1.411],
            [0.06667,1.5], 
            [0.1,1.578],
            [0.1333,1.6456],
            [0.1667,1.703],
            [0.2,1.745],
            [0.2333,1.781],
            [0.2667,1.807],
            [0.3,1.828],
            [0.3333,1.818],
            [0.3667,1.818],
            [0.4,1.807],
            [0.4333,1.776],
            [0.4667,1.734],
            [0.5,1.682],
            [0.5333,1.63],
            [0.567,1.552],
            [0.6,1.469],
            [0.6333,1.37],
            [0.667,1.266],
            [0.7,1.151],
            [0.733,1.026],
            [0.7667,0.875],
            [0.8,0.719],
            [0.8333,0.5573],
            [0.867,0.3854],
            [0.9,0.193],
            [0.9333,0.0052] ] )

In [ ]:
plt.plot(BallData[:,0], BallData[:,1], 'o-')
plt.xlabel('t (s)')
plt.ylabel('y (m)')
plt.show()

What is the acceleration $a$? The acceleration is 

 $$ a= \frac{d^2 y}{dt^2}.$$

Numerically this could be coded as

In [ ]:
def second_derivative(data, i):
    return (data[i+1,1]-2*data[i,1]+data[i-1,1])/(data[i+1,0]-data[i,0])**2

We can make a list of pair to plot the acceleration as a function of time. Note that we do not attempt to calculate the acceleration at i = 0 and i = 29 because this would require data outside the array.

In [ ]:
accel = np.zeros(28)

for i in range(1, 28):
    accel[i] = second_derivative(BallData, i)

We can plot the results

In [ ]:
plt.plot(BallData[1:29, 0], accel)
plt.xlabel('t (s)')
plt.ylabel('a (m/s$^2$)')
plt.show()

We can get the mean acceleration over the experiment

In [ ]:
np.mean(accel)

## Numerical integration: Rectangle rule

The simplest algorithm to estimate numerically a definite integral of this sort

$$\int_a^b f(x) dx$$

is to divide the area between 

$$f(a)$$ and  $$f(b)$$ into 

$n$ rectangles of constant width 

$$\Delta x = (b-a)/n$$

and heights $f(x_0)$, $f(x_2)$, $f(x_3)$, $\ldots$, $f(x_{n-1})$

and to sum the areas of these rectangles. 

This leads to the following approximation called the rectangle rule


$$ \int_a^b f(x) dx \approx [f(x_0)+f(x_2)+f(x_3) + \dots +f(x_{n-1})] \Delta x = \sum_{i=0}^{n-1} f(x_i)\Delta x $$

where
	
$$ x_i=a + i \Delta x $$

Let's create our own function that will achieve this type of numerical integration. I will call this function `NIRect` for Numerical Integration using the Rectangle rule. I want to create this function with the following four arguments

- `f`: the function that we want to integrate numerically
- `a`: the lower limit of integration
- `b`: the upper limit of integration
- `n`: the number of elements (i.e. rectangles) to use for the integration

Your function may look like this:

In [ ]:
def NIRect(f, a, b, n):
    dx = (b-a)/n
    sum = 0
    for i in range(n):
        sum += f(a + i*dx) * dx
   
    return sum

Here's how you can use this function. First define the function that you need to integrate, say 

sin(x) from 0 to $\pi/2$, and then call `NIRect` with the appropriate arguments. Let's use only 10 rectangles, i.e.,

In [ ]:
f = np.sin
a = 0
b = np.pi / 2
n = 10

NIRect(f, a, b, n)

The integral

$$\int_0^{\pi/2} \sin (x) dx$$

 can in fact be integrated analytically and we know that the exact answer should be 

$$ -\cos(\pi/2) + \cos(0) = 1 $$

We see that using the rectangle rule with 10 rectangles returns 0.919403, which is an error of roughly 8% compared to the exact solution which is exactly 

#### 1. Let's try with 100 rectangles

In [ ]:
f = np.sin
a = 0
b = np.pi / 2
n = 100

NIRect(f, a, b, n)

We see that increasing the number of rectangles produces a more accurate results. Yet, it is not 1. In fact, it will never be 1 unless you use an infinite number of rectangles. So, when doing numerical calculations like this there is always the issue of determining when to stop the integration. For example, if we use 10 billion rectangles the computer may take days to perform the integration. There is always a trade off between limits of computational power and accuracy needed for your application.  

Note that we don't need to use the variable `f` for the input function. Here's another example with a function that cannot be integrated analytically

$$ g(x) = \frac{1}{x^2} e^{x^2} $$

In [ ]:
def g(x):
    return 1/x**2 * np.exp(x**2)

a = 1
b = 2
n = 10

NIRect(g, a, b, n)

Notice how I called my function `NIRect` this time with the variable name `g` even though the function is originally defined with the variable `f`. This illustrates the power and flexibility of defining functions. Same can be said with the other arguments. For example `a` and `b` can really be anything else. For example

In [ ]:
def MyFunction(y):
    return 3*y**2 + y

LowerLimit = 2.3
UpperLimit = 10.2
NumRectangles = 100

NIRect(MyFunction, LowerLimit, UpperLimit, NumRectangles)


### Lambda functions

Often, these one-liner functions like `g` and `MyFunction` are defined as lambda functions:

In [ ]:
g = lambda x: 1/x**2 * np.exp(x**2)
a = 1
b = 2
n = 10
NIRect(g, a, b, n)

Note that the values can also be passed directly if you really wanted

In [ ]:
NIRect(lambda y: 3*y**2 + y, 2.3, 10.2, 100)

## Numerical integration: Trapezoidal rule

A more accurate method for solving an integral is to use $n$ trapezoids instead of $n$ rectangles. You should understand where this formulation comes from.

$$\int_a^b f(x) dx \approx \frac{1}{2} f(x_0) + \frac{1}{2} f(x_n)+ \sum_{i=1}^{n-1} f(x_i) \Delta x $$

where as in the rectangle rule

 
 $$\Delta x = (b-a)/n$$

and

 	

$$x_i=a + i \Delta x.$$

Note how although we use $n$ trapezoids the function needs to be evaluated up to 
 $f(x_n)$ whereas in the rectangle rule the function only needed to be evaluated up to 
 $f(x_{n-1})$. Can you tell why?

Here's how this can be coded within a new function that I will call `NITrap`

In [ ]:
def NITrap(f, a, b, n):
    dx = (b-a)/n
    sum = 0.5*(f(a) + f(b)) * dx
    for i in range(1, n):
        sum = sum + f(a + i*dx) * dx
   
    return sum

In [ ]:
f = np.sin
a = 0
b = np.pi / 2
n = 1000

print(NIRect(f, a, b, n))
print(NITrap(f, a, b, n))

From these outputs it is quite clear that the trapezoid rule is more accurate using the same number of trapezoids as the rectangle rule.

## Comparing rectangle rule and trapezoid rule

The goal of this section is to compare visually the error associated with the rectangle and the trapezoidal rule as a function of the number of elements (rectangles or trapezoids) used. This will require some coding.  

The integral to evaluate is

$$\int_a^b f(x) dx $$

where


$$ f(x)= \sin(x) + x^3, \quad a=0, \quad  b =5 $$

##### Exercise to be done as a class.

#### 1. Define $f(x)$ as Python function.

#### 2. Plot this function between  x=-10 and x=10.

#### 3. Calculate the exact value of the integral above (with a=0 and b=5)

#### 4. Evaluate numerically the integral using the rectangle rule with 10 rectangles. 

#### 5. Evaluate numerically the integral using the trapezoid rule with 10 trapezoids.

#### 6. Briefly comment on the answers you obtained in 4. and 5. 

#### 7. The absolute relative error of the numerical integration can be calculated as follow: 

$\Delta$ =|Numerical Solution - Exact Solution|/|Exact Solution|

Code this expression for the relative error and calculate $\Delta$ for the results obtained in 4. and 5. above.

#### 8. Redo the same as in 7. but this time calculate the error when using 100 rectangles or trapezoids to evaluate the integral.

#### 9. It should be clear from your previous calculations that the error reduces significantly by increasing the number of elements and that the trapezoid rule is always more accurate than the rectangle rule.  

What would be illuminating would be to plot the relative error for each algorithms (i.e. rectangle and trapezoid rules) as a function of the number $n$ of elements used to approximate numerically the integral. Before coding this you need to become a little more familiar with NumPy arrays.

First, use the `np.arange` function  to automatically create the array: [1,2,3,4,5,6,7,8,9,10]

Now, starting from this first array and multiplying, create the array: [2,4,6,8,10,12,14,16,18,20]

Instead, try and create the array [2,4,8,16,32,64,128,256,512,1024]

#### 10. Next, the challenge is to create a two column matrix with the first column containing the number $n$ of elements used in the numerical integration and in the second column the relative error $\Delta$.  

(This can be done with or without loops.)



Now you can plot the results of this analysis like this:

#### 11. Redo the same analysis with a graph for the trapezoid rule.

#### 12. Finally compare the results obtained in 11 for the rectangle rule and in 12 for the trapezoid rule on a single graph. Draw a brief conclusion to the results obtained graphically.

Comment: The trapezoidal rule converges much faster than the rectangle rule to the exact solution.